In [38]:
import argparse
import json
import os

In [39]:
import tensorflow as tf
print(tf.__version__)

1.15.0


In [40]:
# just a temporary workaround
# pass this code to the setup.py file of the final module
import sys
_ROOT_DIR = '{0}/gcp/cbidmltsf'.format(os.getenv("HOME"))
sys.path.append(_ROOT_DIR)

In [41]:
from dplstm.data import make_input_fn
from dplstm.model import DPLSTM

In [42]:
# build parameters dictionary for interactive execution
# in batch execution, this dictionary comes from parsed cli arguments

parameters = {
    'model_dir': 'lstm_46',
    'learning_rate': 0.01,
    'num_epochs': 20,  # NOT REQUIRED IN DISTRIBUTED MODE, IT IS OVERRIDDEN BY MAX_TRAIN_STEPS
    'max_train_steps': 1100,
    'train_batch_size': 32,
    'eval_interval': 300,
    'keep_checkpoint_max': 3,
    'eval_steps': None,
    'eval_batch_size': 2**16,
    'start_delay_secs': 600,
    'throttle_secs': 600,
    'train_data_path': '{0}/data/tfrecord/train.tfrecord'.format(_ROOT_DIR),
    'eval_data_path': '{0}/data/tfrecord/val.tfrecord'.format(_ROOT_DIR),
    'test_data_path': '{0}/data/tfrecord/test.tfrecord'.format(_ROOT_DIR),    
}

In [43]:
# support for log files
import logging

# advanced numerical operations
import numpy as np

import tensorflow as tf

_LOG_PATH = '{0}/logs'.format(_ROOT_DIR)
logging.basicConfig(filename='{}/test.log'.format(_LOG_PATH),
                    level=logging.INFO,
                    format='%(asctime)s - %(name)s - %(threadName)s -  %(levelname)s - %(message)s')

logging.info('Logging to {}/test.log.'.format(_LOG_PATH))

# ToDo: pass a complex dictionary as a starting point for the LSTM network chromosome
'''
    Deep/Parallel LSTM network chromosome includes:
    m_hour: [24, 12, 8, etc], tau=1, implement this variable later and work now with m_hour=24
    m_day: [7, 14, etc], tau=7, implement this variable later and work now with m_day=7
    m_week: [4, 8, 12, etc], tau=168, implement this variable later and work now with m_week=4
    hidden_hour:
    hidden_day:
    hidden_week:
    levels_hour:
    levels_day:
    levels_week:
'''

_LOG_PATH = '{0}/logs'.format(_ROOT_DIR)
logging.basicConfig(filename='{}/test.log'.format(_LOG_PATH),
                    level=logging.INFO,
                    format='%(asctime)s - %(name)s - %(threadName)s -  %(levelname)s - %(message)s')

logging.info('Logging to {}/test.log.'.format(_LOG_PATH))

In [44]:
# remove main function and transfer everything to first-level code cells
tf.logging.set_verbosity(tf.logging.INFO)

In [45]:
def time_series_forecaster(features, labels, mode):
    # instantiate network topology from the corresponding class
    forecaster_topology = DPLSTM()

    # ToDo: global_step might be moved to TRAIN scope
    global_step = tf.train.get_global_step()

    # call operator to forecaster_topology, over features
    forecast = forecaster_topology(features)

    # predictions are stored in a dictionary for further use at inference stage
    predictions = {
        "forecast": forecast
    }

    # CHANGE MODEL FUNCTION STRUCTURE ACCORDING TO GILLARD'S ARCHITECTURE

    # Estimator in TRAIN or EVAL mode
    if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
        # use labels and predictions to define training loss and evaluation loss
        # generate summaries for TensorBoard
        with tf.name_scope('loss'):
            mean_squared_error = tf.losses.mean_squared_error(
                labels=labels, predictions=forecast, scope='loss')
            tf.summary.scalar('loss', mean_squared_error)

        with tf.name_scope('val_loss'):
            val_loss = tf.metrics.mean_squared_error(
                labels=labels, predictions=forecast, name='mse')
            tf.summary.scalar('val_loss', val_loss[1])

        # this is only required for PREDICT mode
        prediction_hooks = None

        # Estimator in TRAIN mode ONLY
        if mode == tf.estimator.ModeKeys.TRAIN:
            # This is needed for batch normalization, but has no effect otherwise
            update_ops = tf.get_collection(key=tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(control_inputs=update_ops):
                train_op = tf.contrib.layers.optimize_loss(
                    loss=mean_squared_error,
                    global_step=global_step,
                    learning_rate=parameters['learning_rate'],
                    optimizer="Adam")
            # Create a hook to print acc, loss & global step every 100 iter
            train_hook_list = []
            train_tensors_log = {'val_loss': val_loss[1],
                                 'loss': mean_squared_error,
                                 'global_step': global_step}
            train_hook_list.append(tf.train.LoggingTensorHook(
                tensors=train_tensors_log, every_n_iter=100))

            predictions = None  # this is not required in TRAIN mode
            loss = mean_squared_error
            eval_metric_ops = None
            training_hooks = train_hook_list
            evaluation_hooks = None

        else:  # Estimator in EVAL mode ONLY
            loss = mean_squared_error
            train_op = None
            training_hooks = None
            eval_metric_ops = {'val_loss': val_loss}
            evaluation_hooks = None

    # Estimator in PREDICT mode ONLY
    else:
        loss = None
        train_op = None
        eval_metric_ops = None
        training_hooks = None
        evaluation_hooks = None
        prediction_hooks = None  # this might change as we are in PREDICT mode

    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=predictions,
        loss=loss,
        train_op=train_op,
        eval_metric_ops=eval_metric_ops,
        # export_outputs=not_used_yet (for TensorFlow Serving, redirected from predictions if omitted)
        # training_chief_hooks=not_used_yet
        # ToDo: verify use of training_hooks
        # temporarily disable training hooks
        # training_hooks=training_hooks,
        training_hooks=training_hooks,
        # scaffold=not_used_yet
        evaluation_hooks=evaluation_hooks,
        prediction_hooks=prediction_hooks
    )

# ToDo: evaluate the convenience of packaging execution in a tf.app
# in the meantime, run the estimator outside tf.app package
# tf.logging.set_verbosity(tf.logging.INFO)

In [46]:
# ensure file writer cache is clear for TensorBoard events file
tf.summary.FileWriterCache.clear()

In [47]:
# parameters required for RunConfig()
# _EVAL_INTERVAL = 300  # how often checkpoints are written out, given in seconds
# _KEEP_CHECKPOINT_MAX = 3  # how many checkpoints to keep
# _MAX_TRAIN_STEPS = 16000
# _TRAIN_BATCH_SIZE = 2**5
# _EVAL_STEPS = None  # if None, evaluate on entire dataset
# _EVAL_BATCH_SIZE = 2**16
# _START_DELAY_SECONDS = 600
# _THROTTLE_SECONDS = 600

In [48]:
# instantiate base estimator class for custom model function
tsf_estimator = tf.estimator.Estimator(
    model_fn=time_series_forecaster,
    # no parameters passed at this point
    # params=hparams,
    # parameters passed in original model include: train_data_path, batch_size, augment, train_steps
    config=tf.estimator.RunConfig(
        save_checkpoints_secs=parameters['eval_interval'],
        keep_checkpoint_max=parameters['keep_checkpoint_max']
    ),
    model_dir='{0}/dplstm/{1}'.format(_ROOT_DIR, parameters['model_dir']))

INFO:tensorflow:Using config: {'_task_id': 0, '_num_ps_replicas': 0, '_protocol': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_save_summary_steps': 100, '_eval_distribute': None, '_train_distribute': None, '_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 300, '_session_creation_timeout_secs': 7200, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4d6427b978>, '_num_worker_replicas': 1, '_master': '', '_model_dir': '/home/jupyter/gcp/cbidmltsf/dplstm/lstm_46', '_save_checkpoints_steps': None, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_device_fn': None, '_is_chief': True, '_keep_checkpoint_max': 3, '_global_id_in_cluster': 0, '_task_type': 'worker', '_evaluation_master': '', '_tf_random_seed': None}


In [49]:
# Set estimator's train_spec to use train_input_fn and train for so many steps
train_spec = tf.estimator.TrainSpec(
    input_fn=make_input_fn(
        tfrecord_path=parameters['train_data_path'],
        batch_size=parameters['train_batch_size'],
        mode=tf.estimator.ModeKeys.TRAIN
    ),
    max_steps=parameters['max_train_steps'])

In [50]:
# ToDo: activate TensorFlow Serving for distributed inference
# Create exporter that uses serving_input_fn to create saved_model for serving
# exporter = tf.estimator.LatestExporter(
#     name="exporter",
#     serving_input_receiver_fn=serving_input_fn)

In [51]:
# now the customized input function for serving predictions on dplstm
# target dictionary is {'hourly': [?, 24, 1], 'daily': [?, 7, 1], 'weekly': [?, 4, 1]}
feature_placeholders = {
    'hourly': tf.placeholder(tf.float32, [None, 24]),
    'daily': tf.placeholder(tf.float32, [None, 7]),
    'weekly': tf.placeholder(tf.float32, [None, 4])
}

In [52]:
features = {
    key: tf.expand_dims(tensor, -1)
    for key, tensor in feature_placeholders.items()
}

In [53]:
def serving_input_fn():

    feature_placeholders = {
    'hourly': tf.placeholder(tf.float32, [None, 24]),
    'daily': tf.placeholder(tf.float32, [None, 7]),
    'weekly': tf.placeholder(tf.float32, [None, 4])
    }
    
    features = {
    key: tf.expand_dims(tensor, -1)
    for key, tensor in feature_placeholders.items()
    }
    
    return tf.estimator.export.ServingInputReceiver(features = features, receiver_tensors = feature_placeholders)

In [54]:
# create exporter that uses serving_input_fn to create saved_model for serving
exporter = tf.estimator.LatestExporter(
    name = "exporter", 
    serving_input_receiver_fn = serving_input_fn)

In [55]:
# Set estimator's eval_spec to use eval_input_fn and export saved_model
eval_spec = tf.estimator.EvalSpec(
    input_fn=make_input_fn(
        tfrecord_path=parameters['eval_data_path'],
        batch_size=parameters['eval_batch_size'],
        mode=tf.estimator.ModeKeys.EVAL
    ),
    steps=parameters['eval_steps'],  # use None to evaluate on the entire dataset
    exporters=exporter,
    start_delay_secs=parameters['start_delay_secs'],  # delay first evaluation
    throttle_secs=parameters['throttle_secs']  # evaluate at a different rate (usually longer) than checkpoint
)

In [56]:
# Run train_and_evaluate loop
tf.estimator.train_and_evaluate(
    estimator=tsf_estimator,
    train_spec=train_spec,
    eval_spec=eval_spec)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 300.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
The TensorFlow contrib module will not be inc

({'global_step': 1100, 'loss': 0.0054646125, 'val_loss': 0.0054646125},
 [b'/home/jupyter/gcp/cbidmltsf/dplstm/lstm_46/export/exporter/1575999035'])

In [66]:
# now the final step... predict using the saved model over TFRecord test dataset
# first, locate and load the saved model
_LAST_EXPORTER = '1575999035'
_SAVED_MODEL_DIR = '{0}/dplstm/{1}/export/exporter/{2}'.format(_ROOT_DIR, parameters['model_dir'], _LAST_EXPORTER)

In [67]:
# build a prediction function
predict_fn = tf.contrib.predictor.from_saved_model(_SAVED_MODEL_DIR)

INFO:tensorflow:Restoring parameters from /home/jupyter/gcp/cbidmltsf/dplstm/lstm_46/export/exporter/1575999035/variables/variables


In [75]:
# now locate TFRecord test dataset and load it, need to parse the examples to a dictionary
test_dataset_filename = '{0}/data/tfrecord/test.tfrecord'.format(_ROOT_DIR)
test_dataset_filename

'/home/jupyter/gcp/cbidmltsf/data/tfrecord/test.tfrecord'

In [72]:
# now read the dataset from TFRecord file using non-deprecated methods from tf.data module
test_raw_dataset = tf.data.TFRecordDataset(test_dataset_filename)
test_raw_dataset

<TFRecordDatasetV1 shapes: (), types: tf.string>

In [73]:
test_raw_dataset.output_types

Instructions for updating:
Use `tf.compat.v1.data.get_output_types(dataset)`.


tf.string

In [74]:
test_raw_dataset.output_shapes

Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.


TensorShape([])

In [78]:
sess = tf.InteractiveSession()

In [112]:
# can I access to the binary, string-based, raw dataset using a one-shot iterator?
iterator = test_raw_dataset.make_one_shot_iterator()
next_element = iterator.get_next()

# there is only one row in the raw dataset
value = sess.run(next_element)
print(value)

b'\n\xcf\x01\n\x1e\n\x06weekly\x12\x14\x12\x12\n\x10\x15(\xda>\xd2y\xf4>\x1b\x11\x03?7\xa7\x0b?\n\x12\n\x06target\x12\x08\x12\x06\n\x04W\xcb\xfc>\n)\n\x05daily\x12 \x12\x1e\n\x1c7\xa7\x0b?\x85\x9a\x0c?\x81\xf5\x05?sO\xd6>\x9d\xaa\xcd>\x81y\xc6>\x01\x92\x15?\nn\n\x06hourly\x12d\x12b\n`\x01\x92\x15?\xcbo\x1f?\xe4A/?\xc4\xcc(?\xe1\x961?\xce\x00=?\xee\xbcJ?d\xc1L?\xea\xe2+?\xff+"?D\x93\xed>G\xc4\x98>\xc4d\xa5>\xbc\xd4\x85>\\Og>\x80\xf9\x8f>\x9d\x8ep>\x9f\xeb\x8d>I\xc2\x88>[b^>\xb8\xf8)>N\xff\xbb>6\xb0\xd4>\x91H\xf0>'


In [162]:
value

b'\n\xcf\x01\n\x1e\n\x06weekly\x12\x14\x12\x12\n\x10\x15(\xda>\xd2y\xf4>\x1b\x11\x03?7\xa7\x0b?\n\x12\n\x06target\x12\x08\x12\x06\n\x04W\xcb\xfc>\n)\n\x05daily\x12 \x12\x1e\n\x1c7\xa7\x0b?\x85\x9a\x0c?\x81\xf5\x05?sO\xd6>\x9d\xaa\xcd>\x81y\xc6>\x01\x92\x15?\nn\n\x06hourly\x12d\x12b\n`\x01\x92\x15?\xcbo\x1f?\xe4A/?\xc4\xcc(?\xe1\x961?\xce\x00=?\xee\xbcJ?d\xc1L?\xea\xe2+?\xff+"?D\x93\xed>G\xc4\x98>\xc4d\xa5>\xbc\xd4\x85>\\Og>\x80\xf9\x8f>\x9d\x8ep>\x9f\xeb\x8d>I\xc2\x88>[b^>\xb8\xf8)>N\xff\xbb>6\xb0\xd4>\x91H\xf0>'

In [167]:
value[0:10]

b'\n\xcf\x01\n\x1e\n\x06wee'

In [80]:
# features dictionary for parsing TFrecord files
read_features = {
    'hourly': tf.io.VarLenFeature(dtype=tf.float32),
    'daily': tf.io.VarLenFeature(dtype=tf.float32),
    'weekly': tf.io.VarLenFeature(dtype=tf.float32),
    'target': tf.io.FixedLenFeature([], dtype=tf.float32)}

In [170]:
# features dictionary for parsing TFrecord files as numpy arrays
read_features_as_numpy = {
    'hourly': tf.io.VarLenFeature(dtype=np.float32),
    'daily': tf.io.VarLenFeature(dtype=np.float32),
    'weekly': tf.io.VarLenFeature(dtype=np.float32),
    'target': tf.io.FixedLenFeature([], dtype=np.float32)}

In [172]:
# another parsing function with no target
def _parse_dataset_function_no_target_as_numpy(example_proto, objective_shape):
    # parse the input tf.Example proto using the dictionary called read_features (above defined)
    row = tf.io.parse_single_example(example_proto, read_features_as_numpy)
    # pass objective shape as a list of lists [hourly_shape, daily_shape, weekly_shape]
    hourly = tf.reshape(row['hourly'].values, objective_shape[0])
    daily = tf.reshape(row['daily'].values, objective_shape[1])
    weekly = tf.reshape(row['weekly'].values, objective_shape[2])
    return {'hourly': hourly, 'daily': daily, 'weekly': weekly}

In [174]:
parsed_dataset_as_numpy = test_raw_dataset.map(lambda row: _parse_dataset_function_no_target_as_numpy(row, [[24, 1], [7, 1], [4, 1]]))

In [176]:
parsed_dataset_as_numpy.output_types

{'daily': tf.float32, 'hourly': tf.float32, 'weekly': tf.float32}

In [81]:
def _parse_dataset_function(example_proto, objective_shape):
    # parse the input tf.Example proto using the dictionary called read_features (above defined)
    row = tf.io.parse_single_example(example_proto, read_features)
    # pass objective shape as a list of lists [hourly_shape, daily_shape, weekly_shape]
    hourly = tf.reshape(row['hourly'].values, objective_shape[0])
    daily = tf.reshape(row['daily'].values, objective_shape[1])
    weekly = tf.reshape(row['weekly'].values, objective_shape[2])
    target = tf.reshape(row['target'], [1, ])
    return {'hourly': hourly, 'daily': daily, 'weekly': weekly}, target

In [168]:
# another parsing function with no target
def _parse_dataset_function_no_target(example_proto, objective_shape):
    # parse the input tf.Example proto using the dictionary called read_features (above defined)
    row = tf.io.parse_single_example(example_proto, read_features)
    # pass objective shape as a list of lists [hourly_shape, daily_shape, weekly_shape]
    hourly = tf.reshape(row['hourly'].values, objective_shape[0])
    daily = tf.reshape(row['daily'].values, objective_shape[1])
    weekly = tf.reshape(row['weekly'].values, objective_shape[2])
    return {'hourly': hourly, 'daily': daily, 'weekly': weekly}

In [161]:
parsed_dataset.batch(batch_size=1).make_one_shot_iterator().get_next()

{'daily': <tf.Tensor 'IteratorGetNext_7:0' shape=(?, 7, 1) dtype=float32>,
 'hourly': <tf.Tensor 'IteratorGetNext_7:1' shape=(?, 24, 1) dtype=float32>,
 'weekly': <tf.Tensor 'IteratorGetNext_7:2' shape=(?, 4, 1) dtype=float32>}

In [179]:
parsed_dataset_as_numpy.make_one_shot_iterator().get_next()

{'daily': <tf.Tensor 'IteratorGetNext_8:0' shape=(7, 1) dtype=float32>,
 'hourly': <tf.Tensor 'IteratorGetNext_8:1' shape=(24, 1) dtype=float32>,
 'weekly': <tf.Tensor 'IteratorGetNext_8:2' shape=(4, 1) dtype=float32>}

In [181]:
predict_fn(parsed_dataset_as_numpy.make_one_shot_iterator().get_next())

TypeError: The value of a feed cannot be a tf.Tensor object. Acceptable feed values include Python scalars, strings, lists, numpy ndarrays, or TensorHandles. For reference, the tensor object was Tensor("IteratorGetNext_10:1", shape=(24, 1), dtype=float32) which was passed to the feed with key Tensor("Placeholder:0", shape=(?, 24), dtype=float32).

In [149]:
import numpy as np

In [152]:
.shape

(1, 24)

In [153]:
# let's try with a custom-generated dictionary
fake_parsed_row = {
    'hourly': np.zeros([1, 24]),
    'daily': np.zeros([1, 7]),
    'weekly': np.zeros([1, 4])
}

In [157]:
fake_parsed_row['hourly'].shape

(1, 24)

In [158]:
# should I use the prediction function over the serialized examples or over the parsed dataset?
predict_fn(fake_parsed_row)

{'forecast': array([[0.15336634]], dtype=float32)}

In [103]:
# let's look what happens in Zhang's tutorial
import pandas as pd

In [104]:
# test inputs represented by a Pandas DataFrame.
inputs = pd.DataFrame({
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
})

In [105]:
# convert input data into serialized Example strings.
examples = []
for index, row in inputs.iterrows():
    feature = {}
    for col, value in row.iteritems():
        feature[col] = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
    example = tf.train.Example(
        features=tf.train.Features(
            feature=feature
        )
    )
    examples.append(example.SerializeToString())

In [128]:
examples

[b'\nb\n\x16\n\nSepalWidth\x12\x08\x12\x06\n\x0433S@\n\x17\n\x0bSepalLength\x12\x08\x12\x06\n\x0433\xa3@\n\x17\n\x0bPetalLength\x12\x08\x12\x06\n\x04\x9a\x99\xd9?\n\x16\n\nPetalWidth\x12\x08\x12\x06\n\x04\x00\x00\x00?',
 b'\nb\n\x17\n\x0bSepalLength\x12\x08\x12\x06\n\x04\xcd\xcc\xbc@\n\x17\n\x0bPetalLength\x12\x08\x12\x06\n\x04ff\x86@\n\x16\n\nPetalWidth\x12\x08\x12\x06\n\x04\x00\x00\xc0?\n\x16\n\nSepalWidth\x12\x08\x12\x06\n\x04\x00\x00@@',
 b'\nb\n\x16\n\nSepalWidth\x12\x08\x12\x06\n\x04ffF@\n\x17\n\x0bSepalLength\x12\x08\x12\x06\n\x04\xcd\xcc\xdc@\n\x17\n\x0bPetalLength\x12\x08\x12\x06\n\x04\xcd\xcc\xac@\n\x16\n\nPetalWidth\x12\x08\x12\x06\n\x04ff\x06@']

In [37]:
# predict based on checkpoints and plot results with Bokeh

In [38]:
# Anaconda Interactive Visualization
from bokeh.plotting import figure
from bokeh.plotting import output_file, save

In [39]:
# persistence for the scaler located in $DATA/scalers
from sklearn.externals import joblib

# reload scaler fitted model here
scaler = joblib.load('{0}/data/scalers/ci_LSTM_scaler.save'.format(_ROOT_DIR))

/usr/local/lib/python3.5/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.19.1 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [40]:
# build predictions using the trained model
# ToDo: use TensorFlow Serving instead of the custom estimator
# pred = list(tsf_estimator.predict(input_fn=test_input_fn))
pred = list(
    tsf_estimator.predict(
        input_fn=make_input_fn(
            tfrecord_path=parameters['test_data_path'],
            batch_size=32,
            mode=tf.estimator.ModeKeys.PREDICT
        )
    )
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/jupyter/gcp/cbidmltsf/dplstm/lstm_46/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [41]:
len(pred)

566

In [43]:
# let's look at a predictions subset
pred[:10]

[{'forecast': array([0.4926325], dtype=float32)},
 {'forecast': array([0.54299], dtype=float32)},
 {'forecast': array([0.5770303], dtype=float32)},
 {'forecast': array([0.58423465], dtype=float32)},
 {'forecast': array([0.6234549], dtype=float32)},
 {'forecast': array([0.6371721], dtype=float32)},
 {'forecast': array([0.6415772], dtype=float32)},
 {'forecast': array([0.6419458], dtype=float32)},
 {'forecast': array([0.6172749], dtype=float32)},
 {'forecast': array([0.56820893], dtype=float32)}]

In [44]:
# move to array and re-scale
pred = [p['forecast'][0] for p in pred]
pred = np.asarray(pred)
pred_ci = scaler.inverse_transform(pred.reshape(-1, 1))
pred_ci = np.squeeze(pred_ci)

In [51]:
pred_ci[:10]

array([4.056008 , 4.470618 , 4.7508826, 4.810199 , 5.1331124, 5.2460504,
       5.2823186, 5.2853537, 5.0822296, 4.6782537], dtype=float32)

In [46]:
# ToDo: get ytarget_test array from test.tfrecord dataset to perform the following operation
# temporarily get the array from disk
y_test = np.load('{0}/data/arrays/y_test.npy'.format(_ROOT_DIR))
n = 0  # first step ahead
ytarget_test = y_test[:, n]
ytarget_test = ytarget_test.reshape(ytarget_test.shape[0], 1)

actual_ci = scaler.inverse_transform(ytarget_test)
actual_ci = np.squeeze(actual_ci)

In [48]:
_EQUIPMENT = 'CPE04105'

In [49]:
ci_predictions_fig = figure(title='Predicted Current Imbalance for ' + _EQUIPMENT,
                            background_fill_color='#E8DDCB',
                            plot_width=1800, plot_height=450, x_axis_type='datetime')

# ToDo: yts_test array is required to get timestamps for plot, then wire it now and get it from TFRecord later...
yts_test = np.load('{0}/data/arrays/yts_test.npy'.format(_ROOT_DIR), allow_pickle=True)

ci_predictions_fig.line(yts_test[:, n],
                        actual_ci, line_color='red',
                        line_width=1, alpha=0.7, legend='Actual')

ci_predictions_fig.line(yts_test[:, n],
                        pred_ci, line_color='blue',
                        line_width=1, alpha=0.7, legend='Predicted')

ci_predictions_fig.legend.location = "top_right"
ci_predictions_fig.legend.background_fill_color = "darkgrey"

ci_predictions_fig.xaxis.axis_label = 'Timestamp'
ci_predictions_fig.yaxis.axis_label = 'Current Imbalance [%]'

# output_file('{0}/plots/'.format(_ROOT_DIR) + '{:03d}'.format(n) + '.html', title=_EQUIPMENT)
output_file('{0}/plots/'.format(_ROOT_DIR) + '{}'.format(parameters['model_dir']) + '.html', title=_EQUIPMENT)
save(ci_predictions_fig)

'/home/jupyter/gcp/cbidmltsf/plots/lstm_46.html'

In [4]:
# starting from here, just open the test dataset and use it for prediction
# also, verify the cli tool for predictions, as described in Zhang, 2019
# saved_model_cli show --dir export/1524906774 --tag_set serve --signature_def serving_default

In [12]:
# take a look at the saved model, on the terminal
'''
$ saved_model_cli show --dir 1575917715 --tag_set serve --signature_def serving_default
The given SavedModel SignatureDef contains the following input(s):
  inputs['daily'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 7)
      name: Placeholder_1:0
  inputs['hourly'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 24)
      name: Placeholder:0
  inputs['weekly'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 4)
      name: Placeholder_2:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['forecast'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: output_4_1/Sigmoid:0
Method name is: tensorflow/serving/predict
'''

# It works! Now it's time to test it

"\n$ saved_model_cli show --dir 1575917715 --tag_set serve --signature_def serving_default\nThe given SavedModel SignatureDef contains the following input(s):\n  inputs['daily'] tensor_info:\n      dtype: DT_FLOAT\n      shape: (-1, 7)\n      name: Placeholder_1:0\n  inputs['hourly'] tensor_info:\n      dtype: DT_FLOAT\n      shape: (-1, 24)\n      name: Placeholder:0\n  inputs['weekly'] tensor_info:\n      dtype: DT_FLOAT\n      shape: (-1, 4)\n      name: Placeholder_2:0\nThe given SavedModel SignatureDef contains the following output(s):\n  outputs['forecast'] tensor_info:\n      dtype: DT_FLOAT\n      shape: (-1, 1)\n      name: output_4_1/Sigmoid:0\nMethod name is: tensorflow/serving/predict\n"